In [1]:
from os import listdir
from os.path import isfile, join
import pickle

%run ./generator.ipynb
%run ./variables.ipynb
%run ./utils.ipynb

AttributeError: 'NoneType' object has no attribute 'shape'

In [2]:
images = [f for f in listdir(DATASET_PATH) if isfile(join(DATASET_PATH, f))]
#print(images)

tmp_code = {}
i = 1
for file in images:
    taxon = file.split('_')[1]
    if not (taxon in tmp_code):
        tmp_code[taxon] = i
        i+=1
savePickle(tmp_code, "tmp", "ids", 0)

In [ ]:
n_id = 0
for i in range(N_IMAGES):
    final_img, labels = Generator_Img(DATASET_PATH = DATASET_PATH, simple_angles = True, size_px = 1000, fast=True, verbose=False)
    saveImg(final_img, "tmp", "png_img", n_id)
    savePickle(labels, "tmp", "labels", n_id)
    print(n_id,"/",N_IMAGES)
    n_id += 1

0 / 200
1 / 200
2 / 200
3 / 200
4 / 200
5 / 200
6 / 200
7 / 200
8 / 200
9 / 200
10 / 200
11 / 200
12 / 200
13 / 200
14 / 200
15 / 200
16 / 200
17 / 200
18 / 200
19 / 200
20 / 200
21 / 200
22 / 200
23 / 200
24 / 200
25 / 200
26 / 200
27 / 200
28 / 200
29 / 200
30 / 200
31 / 200
32 / 200
33 / 200
34 / 200
35 / 200
36 / 200
37 / 200
38 / 200
39 / 200
40 / 200
41 / 200
42 / 200
43 / 200
44 / 200
45 / 200
46 / 200
47 / 200
48 / 200
49 / 200
50 / 200
51 / 200
52 / 200
53 / 200
54 / 200
55 / 200
56 / 200
57 / 200
58 / 200
59 / 200
60 / 200
61 / 200
62 / 200
63 / 200
64 / 200
65 / 200
66 / 200
67 / 200
68 / 200
69 / 200
70 / 200
71 / 200
72 / 200
73 / 200
74 / 200
75 / 200
76 / 200
77 / 200
78 / 200
